In [1]:
# === BẮT ĐẦU ĐOẠN CODE KIỂM TRA MỚI (ROBUST HƠN) ===

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding

# 1. TẠO DỮ LIỆU GIẢ LẬP (KHÔNG THAY ĐỔI)
data = {
    "prompt": [
        "Vào những năm 1960, nơi nào trở thành trung tâm của thế hệ âm nhạc beat và folk?",
        "Tác phẩm điêu khắc nổi tiếng nào nằm ở lối vào công viên?",
    ],
    "context": [
        "Những năm 1960 khu này trở thành một trong những trung tâm của thế hệ âm nhạc beat và folk (dân gian), khi Allen Ginsberg và Bob Dylan sinh sống tại đó.",
        "Công viên có một cái hồ lớn ở trung tâm. Ở lối vào phía nam là một bức tượng ngựa bằng đồng.",
    ],
    "response": [
        "Quảng trường Washington là trung tâm của thế hệ âm nhạc beat và folk vào những năm 1960.",
        "Một bức tượng sư tử bằng đá cẩm thạch được đặt ở lối vào công viên.",
    ],
    "label": ["no", "intrinsic"],
}
df = pd.DataFrame(data)

# 2. CẤU HÌNH TỐI GIẢN (KHÔNG THAY ĐỔI)
MODEL_NAME = "MoritzLaurer/ernie-m-large-mnli-xnli"
MAX_LENGTH = 512
LABEL_MAP = {"intrinsic": 0, "extrinsic": 1, "no": 2}

# 3. CHUẨN BỊ DỮ LIỆU (KHÔNG THAY ĐỔI)
premise = (
    "Câu hỏi: " + df["prompt"].astype(str) + " Ngữ cảnh: " + df["context"].astype(str)
)
hypothesis = df["response"].astype(str)
df["input_text"] = premise + " </s></s> " + hypothesis
df["label_id"] = df["label"].map(LABEL_MAP)

# ( # <-- SỬA ĐỔI 1 ) Tải tokenizer TRƯỚC
print("Đang tải tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# ( # <-- SỬA ĐỔI 2 ) Tokenize TOÀN BỘ dữ liệu cùng một lúc
print("Đang tokenize toàn bộ dữ liệu...")
# Chuyển cột input_text thành một danh sách các câu
list_of_texts = df["input_text"].to_list()
# Gọi tokenizer trên cả danh sách
encodings = tokenizer(
    list_of_texts,
    max_length=MAX_LENGTH,
    truncation=True,
    padding=False,  # Quan trọng: Không padding ở bước này
)


# ( # <-- SỬA ĐỔI 3 ) CLASS DATASET MỚI (ĐƠN GIẢN HƠN)
# Nhiệm vụ của nó bây giờ chỉ là lấy ra các item đã được xử lý sẵn
class PreTokenizedDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Lấy tất cả các encoding cho item thứ idx
        item = {key: val[idx] for key, val in self.encodings.items()}
        # Thêm nhãn vào
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


# 5. TẠO PIPELINE (với Dataset mới)
try:
    print("Đang tạo Dataset từ dữ liệu đã tokenize...")
    labels = df["label_id"].to_list()
    dataset = PreTokenizedDataset(encodings=encodings, labels=labels)

    print("Đang tạo DataCollator...")
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    print("Đang tạo DataLoader...")
    loader = DataLoader(dataset, batch_size=2, collate_fn=data_collator)

    print("\n--- BẮT ĐẦU LẤY BATCH ĐẦU TIÊN ---")
    sample_batch = next(iter(loader))
    print("✅✅✅ THÀNH CÔNG! Đã lấy được batch đầu tiên mà không có lỗi.")
    print("Kích thước input_ids:", sample_batch["input_ids"].shape)

except Exception as e:
    import traceback

    print(f"\n❌❌❌ THẤT BẠI! Vẫn xảy ra lỗi.")
    print(f"Loại lỗi: {type(e).__name__}")
    print(f"Thông điệp lỗi: {e}")
    # In traceback để xem chi tiết hơn
    traceback.print_exc()

# === KẾT THÚC ĐOẠN CODE KIỂM TRA ===


Đang tải tokenizer...
Đang tokenize toàn bộ dữ liệu...
Đang tạo Dataset từ dữ liệu đã tokenize...
Đang tạo DataCollator...
Đang tạo DataLoader...

--- BẮT ĐẦU LẤY BATCH ĐẦU TIÊN ---
✅✅✅ THÀNH CÔNG! Đã lấy được batch đầu tiên mà không có lỗi.
Kích thước input_ids: torch.Size([2, 93])
